## Dev Center Setup

This notebook walks through a sample app setup, which consists of the following layers:

* The foundation is the network setup - VPC creation, subnet definitions, internet and NAT gateways, etc.
* ECS cluster, including autoscale setup
* Load balancer configuration, including HTTPS configuration and optional route 53 alias
* Logging set up
* Installation of an application container

### Region and Account

This notebook assumes we are running this in the dev center developer sandbox account.

### Assumptions

Prior to running this set up, the following must be in place.

* The cloud formation templates are available in the s3 bucket referenced in the notebook
* The domain name referenced in the notebook as been obtained and set up in a hosted zone in Route 53
* The certificate associated with the domain is available in AWS ACM
* A bucket containing sumo.zip is available.
* The key pair used for launching EC2 instances has been created.
* The database used by the application has been created.

### Stack Names

In [ ]:
vpc_stack = 'network-demo'
ecs_stack = 'ecs-demo'
alb_stack = 'alb-demo'
hc_stack = 'hc-demo'
sumo_stack = 'sumo-demo'
appsvc_stack = 'pfactors-demo'

### Account Specific Stuff
Set the following according to your set up.

In [ ]:
import os

bucket = os.environ['CF_BUCKET']
sumo_bucket = os.environ['SUMO_BUCKET']
key_name = os.environ['KEYPAIRNAME']
sumo_endpoint = os.environ['SUMO_ENDPOINT']
region = os.environ['AWS_DEFAULT_REGION']

### SDK Setup

In [ ]:
# Note - before starting jupyter notebook set https_proxy, and if needed export
#        AWS_DEFAULT_REGION as well, e.g. AWS_DEFAULT_REGION=us-west-1. This can
#        also be done via AWS config and AWS_DEFAULT_PROFILE

import boto3

client = boto3.client('cloudformation')
waiter = client.get_waiter('stack_create_complete')

## VPC Setup

In [ ]:
response = client.create_stack(
    StackName=vpc_stack,
    TemplateURL='https://s3-' + region + '.amazonaws.com/' + bucket + '/vpc-pub-priv.yaml'
)

print response

print 'waiting for network stack to complete...'
waiter.wait(
    StackName=vpc_stack
)
print 'network stack created'

In [ ]:
# Extract the stack variables we'll need upstream
response = client.describe_stacks(
    StackName=vpc_stack
)

outputs = response['Stacks'][0]['Outputs']
print outputs

vpcId = [d for d in outputs if d['OutputKey'] == 'VpcId'][0]['OutputValue']
privateSubnet1 = [d for d in outputs if d['OutputKey'] == 'PrivateSubnet1'][0]['OutputValue']
privateSubnet2 = [d for d in outputs if d['OutputKey'] == 'PrivateSubnet2'][0]['OutputValue']
publicSubnet1 = [d for d in outputs if d['OutputKey'] == 'PublicSubnet1'][0]['OutputValue']
publicSubnet2 = [d for d in outputs if d['OutputKey'] == 'PublicSubnet2'][0]['OutputValue']

print vpcId

## ECS Cluster Setup

In [ ]:
response = client.create_stack(
    StackName=ecs_stack,
    TemplateURL='https://s3-' + region + '.amazonaws.com/' + bucket + '/ecs-cluster.yaml',
    Parameters=[
        {
            'ParameterKey': 'AMIType',
            'ParameterValue': 'Amazon'
        },
        {
            'ParameterKey': 'EcsClusterName',
            'ParameterValue': 'DemoCluster'
        },
        {
            'ParameterKey': 'KeyName',
            'ParameterValue': key_name
        },
        {
            'ParameterKey': 'MaxSize',
            'ParameterValue': '6'
        },
        {
            'ParameterKey': 'VpcId',
            'ParameterValue': vpcId
        },
        {
            'ParameterKey': 'PrivateSubnet1',
            'ParameterValue': privateSubnet1
        },
        {
            'ParameterKey': 'PrivateSubnet2',
            'ParameterValue': privateSubnet2
        }
        
    ],
    Capabilities=[
        'CAPABILITY_IAM',
    ]
)

print response

In [ ]:
# Wait for it...
waiter = client.get_waiter('stack_create_complete')
waiter.wait(
    StackName=ecs_stack
)

In [ ]:
# Extract the stack outputs we'll need later
response = client.describe_stacks(
    StackName=ecs_stack
)

ecs_outputs = response['Stacks'][0]['Outputs']
print ecs_outputs

ecsStackSecurityGroup = [d for d in ecs_outputs if d['OutputKey'] == 'AppServerSecurityGroupID'][0]['OutputValue']
ecsCluster = [d for d in ecs_outputs if d['OutputKey'] == 'ECSCluster'][0]['OutputValue']

## Load Balancer Setup

For the load balancer set up we need to reference a certificate that was imported into the AWS
certificate manager.

In [ ]:
acmClient = boto3.client('acm')
response = acmClient.list_certificates()
print response, '\n'

certificateArn = ''

for c in response['CertificateSummaryList']:
    if c['DomainName'] == 'elb.amazonaws.com':
        certificateArn = c['CertificateArn']
        
print 'certificate arn: ', certificateArn

In [ ]:
response = client.create_stack(
    StackName=alb_stack,
    TemplateURL='https://s3-' + region + '.amazonaws.com/' + bucket + '/alb.yaml',
    Parameters=[
        {
            'ParameterKey': 'CertificateArn',
            'ParameterValue': certificateArn,
        },
        {
            'ParameterKey': 'LoadBalancerName',
            'ParameterValue': 'alice'
        },
        {
            'ParameterKey': 'VpcId',
            'ParameterValue': vpcId
        },
        {
            'ParameterKey': 'AppServerSecurityGroupID',
            'ParameterValue': ecsStackSecurityGroup
        },
        {
            'ParameterKey': 'PublicSubnet1',
            'ParameterValue': publicSubnet1
        },
        {
            'ParameterKey': 'PublicSubnet2',
            'ParameterValue': publicSubnet2
        }  
    ]
)

print response

In [ ]:
# Wait for it...
print 'waiting for alb...'
waiter.wait(
    StackName=alb_stack
)
print 'alb stack create finished'

In [ ]:
# Get the stack output we'll refer to later downstream
response = client.describe_stacks(
    StackName=alb_stack
)

alb_outputs = response['Stacks'][0]['Outputs']
loadBalancerDNSName = [d for d in alb_outputs if d['OutputKey'] == 'ALBDnsName'][0]['OutputValue']
loadBalancerARN =    [d for d in alb_outputs if d['OutputKey'] == 'ALBListenerArn'][0]['OutputValue'] 


## Install Health Check Service

In [ ]:
# Create the record set stack
response = client.create_stack(
    StackName=hc_stack,
    TemplateURL='https://s3-' + region + '.amazonaws.com/' + bucket + '/hc.yaml',
    Parameters=[
        {
            'ParameterKey': 'VpcId',
            'ParameterValue': vpcId
        },
        {
            'ParameterKey': 'ECSCluster',
            'ParameterValue': ecsCluster
        },
        {
            'ParameterKey': 'ALBListenerArn',
            'ParameterValue': loadBalancerARN
        },
        {
            'ParameterKey':'Registry',
            'ParameterValue':'docker.io'
        }
    ],
    Capabilities=[
        'CAPABILITY_NAMED_IAM',
    ]
)

print response

In [ ]:
print 'waiting for hc stack...'
waiter.wait(
    StackName=hc_stack
)
print 'hc stack creation complete'

### GET using the Alias

In [ ]:
# Grab the proxy config from the environment
import os
proxy = os.environ['https_proxy']

print proxy

In [ ]:
# Need the load balancer endpoint, which is an output of the alb stack
hcEndpoint = 'https://' + loadBalancerDNSName + '/hcping'
print hcEndpoint

In [ ]:
%%bash -s "$hcEndpoint"
curl -i -k $1 

## Configure Sumo Log Writer

The code we package and deploy as containers are 12 factor apps that write their
log streams to standard out. With AWS ECS, we use the aws-log logging stack to
capture log streams as Cloud Watch logs, then set up subscriptions to 
send those log steams to a lambda function.

In [ ]:
# Create the record set stack
response = client.create_stack(
    StackName=sumo_stack,
    TemplateURL='https://s3-' + region + '.amazonaws.com/' +  bucket + '/sumolambda.yaml',
    Parameters=[
        {
            'ParameterKey': 'SumoEndpoint',
            'ParameterValue': sumo_endpoint,
        },
        {
            'ParameterKey': 'CodeFileName',
            'ParameterValue': 'sumo.zip'
        },
        {
            'ParameterKey': 'CodeBucketName',
            'ParameterValue': sumo_bucket
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM',
    ],
)

print response

In [ ]:
waiter.wait(
    StackName=sumo_stack
)
print 'sumo stack created'

In [ ]:
# Extract stack output
# Get the stack output we'll refer to later downstream
response = client.describe_stacks(
    StackName=sumo_stack
)

print response

sumo_outputs = response['Stacks'][0]['Outputs']
lambdaArn = sumo_outputs[0]['OutputValue']
print lambdaArn

## Install Application Services

We can now install application services. We'll install a simple service that computes prime factors. This is 
a proxy for other services that we would install, manage, and scale using AWS ECS.

In [ ]:
response = client.create_stack(
    StackName=appsvc_stack,
    TemplateURL='https://s3-' + region + '.amazonaws.com/' + bucket + '/pfservice.yaml',
    Parameters=[
        {
            'ParameterKey': 'ALBListenerArn',
            'ParameterValue': loadBalancerARN,
        },
        {
            'ParameterKey': 'ECSCluster',
            'ParameterValue': ecsCluster
        },
        {
            'ParameterKey': 'VpcId',
            'ParameterValue': vpcId
        },
        {
            'ParameterKey': 'LambdaArn',
            'ParameterValue': lambdaArn
        }
    ],
    Capabilities=[
        'CAPABILITY_NAMED_IAM',
    ]
)

print response

In [ ]:
waiter.wait(
    StackName=appsvc_stack
)

In [ ]:
pfEndpoint = 'https://' + loadBalancerDNSName + '/pf/1337'
print pfEndpoint

In [ ]:
%%bash -s "$pfEndpoint"
curl -i -k $1


## Tear It Down

In [ ]:
# Tear down the app container stack
waiter = client.get_waiter('stack_delete_complete')
response = client.delete_stack(
    StackName=appsvc_stack
)

print response
waiter.wait(
    StackName=appsvc_stack
)

In [ ]:
# Tear down the Sumo Stack
response = client.delete_stack(
    StackName=sumo_stack
)

print response
waiter.wait(
    StackName=sumo_stack
)

In [ ]:
# Remove the hc stack
response = client.delete_stack(
    StackName=hc_stack
)

print response
waiter.wait(
    StackName=hc_stack
)

In [ ]:
# Remove the alb
response = client.delete_stack(
    StackName=alb_stack
)

print response
waiter.wait(
    StackName=alb_stack
)

In [ ]:
# Remove the ECS cluster - note this usually fails on the first try
# while the cluster instances are draining. I've opened a support case...
response = client.delete_stack(
    StackName=ecs_stack
)

print response
waiter.wait(
    StackName=ecs_stack
)

In [ ]:
# Nuken the netwerk stak
response = client.delete_stack(
    StackName=vpc_stack
)

print response
waiter.wait(
    StackName=vpc_stack
)